## Graph Neural Networks with Pytorch
## Target: ClusterGCN
- Original Paper: https://arxiv.org/abs/1905.07953
- Original Code: https://github.com/rusty1s/pytorch_geometric/blob/master/examples/cluster_gcn_ppi.py

In [ ]:
import os
import sys

import torch
import torch.nn.functional as F
from torch_geometric.datasets import PPI
from torch_geometric.nn import SAGEConv, BatchNorm
from torch_geometric.data import Batch, ClusterData, ClusterLoader, DataLoader
from sklearn.metrics import f1_score

sys.path.append('../')
from utils import *
logger = make_logger(name='clustergcn_logger')

# Load Dataset
path = os.path.join(os.getcwd(), 'data', 'PPI')
train_dataset = PPI(path, split='train')
val_dataset = PPI(path, split='val')
test_dataset = PPI(path, split='test')

logger.info(f"Save Directory: {train_dataset.processed_dir}")

In [ ]:
train_data = Batch.from_data_list(train_dataset)

# adj of cluster_data: cluster_data.data.adj
# you can check partition information in cluster_data.partptr
cluster_data = ClusterData(
    train_data, num_parts=50, recursive=False, save_dir=train_dataset.processed_dir)
train_loader = ClusterLoader(
    cluster_data, batch_size=1, shuffle=True, num_workers=12)

val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

print(cluster_data)

In [ ]:

# Define Model
class ClusterGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers):
        super(ClusterGCN, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        self.batch_norms.append(BatchNorm(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
            self.batch_norms.append(BatchNorm(hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def forward(self, x, edge_index):
        for conv, batch_norm in zip(self.convs[:-1], self.batch_norms):
            x = conv(x, edge_index)
            x = batch_norm(x)
            x = F.relu(x)
            x = F.dropout(x, p=0.2, training=self.training)
        return self.convs[-1](x, edge_index)

**Cluster-GCN**은 **Classic GCN**, **GraphSAGE**, **FastGCN**, **VR-GCN** 등에서 나타난 속도, 메모리 문제 등을 군집화 알고리즘을 통해 논리적으로 풀어낸 방법론이라고 할 수 있다.  

논문 원본은 [이 곳](https://arxiv.org/abs/1905.07953)을 참조하면 좋고, 한국어로 작성한 리뷰 글은 [이 곳](https://greeksharifa.github.io/machine_learning/2021/08/15/ClusterGCN/)에서 확인할 수 있다.  

전체 그래프를 `METIS`라는 군집화 알고리즘을 통해 아래와 같이 복수의 파티션으로 나눈 후,  

$$ \bar{G} = [\mathcal{G}_1, ..., \mathcal{G}_c] = [\{ \mathcal{V}_1, \mathcal{E}_1\}, ...] $$  

각 파티션을 mini-batch로 취급하여 학습을 수행하는 것이 **Cluster-GCN**의 핵심 아이디어이다. 뛰어난 성능을 유지하면서도 속도, 메모리 측면에서도 괄목할만한 성과를 보여준 방법론이라고 할 수 있다.  

`ClusterData`는 군집화를 통해 graph data object를 복수의 subgraph로 나눠주는 역할을 수행한다. `data`에 `torch_geometric.data.Data` 인스턴스를 입력하면 되고 `num_parts` 인자에 원하는 파티션의 수를 입력하면 된다. `save_dir`에서는 나뉜 데이터를 새로 저장할 주소를 입력하면 된다. 기본값은 None이다.  

`ClusterData`의 결과물을 받았다면, 이를 `ClusterLoader`에 넣어주면 위 코드의 `train_loader`에 해당하는 결과물을 얻을 수 있다.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ClusterGCN(in_channels=train_dataset.num_features, hidden_channels=1024,
            out_channels=train_dataset.num_classes, num_layers=6).to(device)
loss_op = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        loss = loss_op(model(data.x, data.edge_index), data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_nodes
    return total_loss / train_data.num_nodes


@torch.no_grad()
def test(loader):
    model.eval()

    ys, preds = [], []
    for data in loader:
        ys.append(data.y)
        out = model(data.x.to(device), data.edge_index.to(device))
        preds.append((out > 0).float().cpu())

    y, pred = torch.cat(ys, dim=0).numpy(), torch.cat(preds, dim=0).numpy()
    return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0


for epoch in range(1, 201):
    loss = train()
    val_f1 = test(val_loader)
    test_f1 = test(test_loader)
    print('Epoch: {:02d}, Loss: {:.4f}, Val: {:.4f}, Test: {:.4f}'.format(
        epoch, loss, val_f1, test_f1))